<div style="text-align: center;">
  <img src="../../../assets/Cover.png" alt="WTF-Agents cover">
</div>

上一节：**[AutoGen框架介绍](../A00-AutoGen_intro/autogen_intro.md)**


### 章节介绍

本节内容我们将讲述如何使用AutoGen从0开始构建一个智能体，内容包括环境的搭建、智能体的构建以及标准内容输出。

注⚠️：AutoGen从[v0.2](https://microsoft.github.io/autogen/0.2/) → v0.4版本进行了一次彻底重构，v0.4采用异步事件驱动架构，旨在解决可观测性、灵活性、交互控制及扩展性等问题，并且与v0.2不兼容。除非另外说明，本教程的内容仅持v0.4及以上版本。

### 1. 环境安装

参考我们在[Anaconda](../../../basics/00-Anaconda/anaconda.md)介绍的关于Anaconda的安装方法，在现有Anaconda环境的基础上，我们搭建一个运行Autogen的基础Python环境，取名为`wtf_agents`。

#### 创建虚拟环境

1. AutoGen要求 Python 3.10 或更高版本，因此我们创建一个 Python 3.12 的虚拟环境。

    ```bash
    conda create -n wtf_agents python=3.12 -y
    ```

2. 创建虚拟环境成功后，激活该虚拟环境。

    ```bash
    conda activate wtf_agents
    ```

3. 安装AutoGen的依赖包。

    ```bash
    pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

    ```

4. 为了能能够执行单元格代码，我们需要安装Jupyter Notebook。

    ```bash
    pip install jupyter ipykernel
    ```

### 2. 创建自己的API私钥
每一个Agent背后都由一个LLM（Large Language Model，大语言模型）驱动，这是它们与用户的交互以及完成指定任务的基础。本教程会采用OpenAI、DeepSeek、Qwen三个平台的LLM作为演示，大家可以根据实际使用情况选择适合的LLM。

####  2.1 创建OpenAI API-Key

1. 登录OpenAI个人私钥网址：https://platform.openai.com/settings/profile/api-keys
2. 点击左下角的`Create new secret key`按钮，创建一个新私钥。
3. 复制生成的API Key，并保存在安全的地方。


<div style="text-align: center;">
  <img src="./assets/apply_OpenAI_personal_apikey.png" alt="Openai personal apikey" style="width: 60%;">
</div>


或者申请一个 `project based API keys`


1. 登录OpenAI个人私钥网址：https://platform.openai.com/settings/organization/api-keys
2. 点击右上角的`Create new secret key`按钮，创建一个新私钥。
3. 复制生成的API Key，并保存在安全的地方。


<div style="text-align: center;">
  <img src="./assets/apply_OpenAI_project_apikey.png" alt="Openai personal apikey" style="width: 60%;">
</div>


####  2.2 创建DeepSeek API-Key
1. 登录DeepSeek Platform：https://platform.deepseek.com/api_keys
2. 点击右上角的`Create new API key`按钮，创建一个新api key。
3. 复制生成的API Key，并保存在安全的地方。

<div style="text-align: center;">
  <img src="./assets/deepseek_apikey.png" alt="Openai personal apikey" style="width: 60%;">
</div>


#### 2.3 创建Qwen API-Key
1. 登录阿里云百炼平台：https://bailian.console.aliyun.com/?tab=model#/api-key
2. 点击右上角的`创建我的API-KEY`按钮，创建一个新api key。
3. 复制生成的API Key，并保存在安全的地方。

<div style="text-align: center;">
  <img src="./assets/Qwen_apikey.png" alt="Openai personal apikey" style="width: 60%;">
</div>


注⚠️：部分平台私钥只会在创建时显示一次，后续无法再次查看，请妥善保管。




#### 2.4 导出API Key到配置文件


在生产环境中，请不要将API Key直接硬编码在代码中，而是应该将其保存在环境变量中，并使用`os.getenv()`函数来获取。以`OPENAI_API_KEY` 为例展示系统变量的设置方法：


**Mac/Linux系统：**

临时设置 (仅对当前窗口有效)，在终端执行命令行：

```bash
export OPENAI_API_KEY="sk-xxx"
```

持久设置 (对整个系统有效)，在终端执行命令行：

```bash
echo "export OPENAI_API_KEY=sk-xxx" >> ~/.bashrc
source ~/.bashrc
```

**Windows系统：**

临时设置 (仅对当前窗口有效)，在终端执行命令行

```bash
set OPENAI_API_KEY=sk-xxx

```

永久设置

```powershell
 setx OPENAI_API_KEY "sk-xxx"
```


### 3. 测试API-Key是否生效

In [ ]:
import os
# 输出 OPENAI_API_KEY  环境变量
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("Qwen_API_KEY"))
print(os.getenv("DeepSeek_API_KEY"))

In [9]:
import os
from openai import OpenAI

# 1. 测试 OpenAI API
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(f"OpenAI Api content:  {response.output_text}")



# 2. 测试 Qwen API
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("Qwen_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    # extra_body={"enable_thinking": False},
)
content = completion.choices[0].message.content
print(f"Qwen Api content:  {content}")

# 测试使用DeepSeek API

client = OpenAI(api_key=os.getenv("DeepSeek_API_KEY"), 
                base_url="https://api.deepseek.com"
                )

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(f"DeepSeek Api content:  {response.choices[0].message.content}")

OpenAI Api content:  Under a shimmering silver moon, a gentle unicorn named Luna sprinkled stardust across a sleepy meadow, wishing every dreamer a night of magical dreams.
Qwen Api content:  我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程等任务。如果你有任何问题或需要帮助，欢迎随时告诉我！
DeepSeek Api content:  Hello! How can I assist you today? 😊


### 4. 接下来我们将会创建自己的第一个智能体🤖，让Ta欢迎你的到来！

#### 4.1 使用OpenAI apikey创建智能体

In [ ]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient  # autogen_ext 是 autogen extension包，提供了对诸多模型的支持。

# 定义一个模型客户端。也可以使用其他实现了 `ChatCompletionClient` 接口的模型客户端。
model_client = OpenAIChatCompletionClient(
    model="gpt-4.1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# 定义一个工具函数，让智能体调用
async def hello_world(name: str) -> str:
    """
    A simple tool that returns a greeting message.
    :param name: The name of the person to greet.

    """
    return f"Hello {name}~ Welcome to WTF-Agent!"


# 定义一个 AssistantAgent，并启用模型、工具、系统消息和反射。
agent = AssistantAgent(
    name="Receptionist",  # The name of the agent.
    model_client=model_client,
    tools=[hello_world],
    system_message="You are a warm receptionist. Please use the tools to help your friends.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# 执行智能体任务，并将输出流式传输到控制台。
async def main() -> None:
    await Console(agent.run_stream(task="Hey~ I am Xiaobang"))
    # Close the connection to the model client.
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()


---------- TextMessage (user) ----------
Hey~ I am Xiaobang
---------- ToolCallRequestEvent (Receptionist) ----------
[FunctionCall(id='call_XlDI1VHANaLnMLsOeBZgD2tP', arguments='{"name":"Xiaobang"}', name='hello_world')]
---------- ToolCallExecutionEvent (Receptionist) ----------
[FunctionExecutionResult(content='Hello Xiaobang~ Welcome to WTF-Agent!', name='hello_world', call_id='call_XlDI1VHANaLnMLsOeBZgD2tP', is_error=False)]
---------- ModelClientStreamingChunkEvent (Receptionist) ----------
Hey Xiaobang~ Welcome! How can I assist you today?


#### 4.2 使用DeepSeek模型实现上述Agent

In [ ]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelFamily


# 创建模型客户端
model_client = OpenAIChatCompletionClient(
    model="deepseek-chat",
    api_key=os.getenv("DeepSeek_API_KEY"),
    base_url="https://api.deepseek.com",
    model_info={
        "vision": False,             # 是否支持图像输入
        "function_calling": True,    # 是否支持函数调用
        "json_output": True,         # 是否支持 JSON 输出
        "family": ModelFamily.UNKNOWN,  # 也可以自定义为 ModelFamily.R1 或其它
        "structured_output": True    # 是否支持结构化输出
    }
)

# 定义一个工具函数，让智能体调用
async def hello_world(name: str) -> str:
    """
    A simple tool that returns a greeting message.
    :param name: The name of the person to greet.

    """
    return f"Hello {name}~ Welcome to WTF-Agent!"

# 定义助手智能体
agent = AssistantAgent(
    name="Receptionist",
    model_client=model_client,
    tools=[hello_world],
    system_message="You are a warm receptionist. Please use the tools to help your friends.",
    reflect_on_tool_use=True,
    model_client_stream=True,
)

# 运行智能体并将消息流式输出到控制台
async def main() -> None:
    await Console(agent.run_stream(task="Hey~ I am Xiaobang"))
    # 关闭与模型客户端的连接
    await model_client.close()

# 运行主函数
await main()

---------- TextMessage (user) ----------
Hey~ I am Xiaobang
---------- ToolCallRequestEvent (Receptionist) ----------
[FunctionCall(id='call_0_58a86283-635b-4dd4-bd59-aad838fcd2fb', arguments='{"name":"Xiaobang"}', name='hello_world')]
---------- ToolCallExecutionEvent (Receptionist) ----------
[FunctionExecutionResult(content='Hello Xiaobang~ Welcome to WTF-Agent!', name='hello_world', call_id='call_0_58a86283-635b-4dd4-bd59-aad838fcd2fb', is_error=False)]
---------- ModelClientStreamingChunkEvent (Receptionist) ----------
Hello Xiaobang~ Welcome to WTF-Agent! How can I assist you today? 😊


### 4.3 最后来我们继续使用Qwen模型实现上述Agent

In [12]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelFamily


# 创建模型客户端
model_client = OpenAIChatCompletionClient(
    model="qwen-plus",
    api_key=os.getenv("Qwen_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model_info={
        "vision": True,             # 是否支持图像输入
        "function_calling": True,    # 是否支持函数调用
        "json_output": True,         # 是否支持 JSON 输出
        "family": ModelFamily.UNKNOWN,  # 也可以自定义为 ModelFamily.R1 或其它
        "structured_output": True    # 是否支持结构化输出
    }
)

# 定义一个工具函数，让智能体调用
async def hello_world(name: str) -> str:
    """
    A simple tool that returns a greeting message.
    :param name: The name of the person to greet.

    """
    return f"Hello {name}~ Welcome to WTF-Agent!"

# 定义助手智能体
agent = AssistantAgent(
    name="Receptionist",
    model_client=model_client,
    tools=[hello_world],
    system_message="You are a warm receptionist. Please use the tools to help your friends.",
    reflect_on_tool_use=True,
    model_client_stream=True,
)

# 运行智能体并将消息流式输出到控制台
async def main() -> None:
    await Console(agent.run_stream(task="Hey~ I am Xiaobang"))
    # 关闭与模型客户端的连接
    await model_client.close()

# 运行主函数
await main()

---------- TextMessage (user) ----------
Hey~ I am Xiaobang
---------- ToolCallRequestEvent (Receptionist) ----------
[FunctionCall(id='call_243f639647c947c1897ccf', arguments='{"name": "Xiaobang"}', name='hello_world')]
---------- ToolCallExecutionEvent (Receptionist) ----------
[FunctionExecutionResult(content='Hello Xiaobang~ Welcome to WTF-Agent!', name='hello_world', call_id='call_243f639647c947c1897ccf', is_error=False)]
---------- ModelClientStreamingChunkEvent (Receptionist) ----------
Hi Xiaobang! It's so nice to meet you. I can see you're new here. How about I help you get settled? Maybe grab you a cup of coffee or show you around our space? Oh, and if you need any tools or resources, just let me know! How are you finding everything so far?


### 5. 查看AutoGen 框架目前已经支持了哪些LLM

我们在定义模型客户端的时候可以看到 `model_info`中有个`family`字段，这个字段表示的是模型所属的LLM,接下来我们查看一下AutoGen目前已经支持的LLM

In [13]:
# 导出 ModelFamily 类
from autogen_core.models import ModelFamily

print("\nModelFamily 属性和方法:")
try:
    model_families = {name: value for name, value in vars(ModelFamily).items() 
                        if not name.startswith('__')}
    if model_families:
        print("模型家族:")
        for name, value in model_families.items():
            print(f"- {name}: {value}")
    else:
        print("未找到模型家族定义")
except Exception as e:
    print(f"获取类变量失败: {e}")


ModelFamily 属性和方法:
模型家族:
- GPT_41: gpt-41
- GPT_45: gpt-45
- GPT_4O: gpt-4o
- O1: o1
- O3: o3
- O4: o4
- GPT_4: gpt-4
- GPT_35: gpt-35
- R1: r1
- GEMINI_1_5_FLASH: gemini-1.5-flash
- GEMINI_1_5_PRO: gemini-1.5-pro
- GEMINI_2_0_FLASH: gemini-2.0-flash
- GEMINI_2_5_PRO: gemini-2.5-pro
- CLAUDE_3_HAIKU: claude-3-haiku
- CLAUDE_3_SONNET: claude-3-sonnet
- CLAUDE_3_OPUS: claude-3-opus
- CLAUDE_3_5_HAIKU: claude-3-5-haiku
- CLAUDE_3_5_SONNET: claude-3-5-sonnet
- CLAUDE_3_7_SONNET: claude-3-7-sonnet
- CODESRAL: codestral
- OPEN_CODESRAL_MAMBA: open-codestral-mamba
- MISTRAL: mistral
- MINISTRAL: ministral
- PIXTRAL: pixtral
- UNKNOWN: unknown
- ANY: typing.Literal['gpt-41', 'gpt-45', 'gpt-4o', 'o1', 'o3', 'o4', 'gpt-4', 'gpt-35', 'r1', 'gemini-1.5-flash', 'gemini-1.5-pro', 'gemini-2.0-flash', 'gemini-2.5-pro', 'claude-3-haiku', 'claude-3-sonnet', 'claude-3-opus', 'claude-3-5-haiku', 'claude-3-5-sonnet', 'claude-3-7-sonnet', 'codestral', 'open-codestral-mamba', 'mistral', 'ministral', 'pixtra

目前（AutoGen v0.5.7 stable 版本）AutoGen支持的LLM信息包括：

> ModelFamily 属性和方法:
>
> 模型家族:
> - GPT_41: gpt-41
> - GPT_45: gpt-45
> - GPT_4O: gpt-4o
> - O1: o1
> - O3: o3
> - O4: o4
> - GPT_4: gpt-4
> - GPT_35: gpt-35
> - R1: r1
> - GEMINI_1_5_FLASH: gemini-1.5-flash
> - GEMINI_1_5_PRO: gemini-1.5-pro
> - GEMINI_2_0_FLASH: gemini-2.0-flash
> - GEMINI_2_5_PRO: gemini-2.5-pro
> - CLAUDE_3_HAIKU: claude-3-haiku
> - CLAUDE_3_SONNET: claude-3-sonnet
> - CLAUDE_3_OPUS: claude-3-opus
> - CLAUDE_3_5_HAIKU: claude-3-5-haiku
> - CLAUDE_3_5_SONNET: claude-3-5-sonnet
> - CLAUDE_3_7_SONNET: claude-3-7-sonnet
> - CODESRAL: codestral
> - OPEN_CODESRAL_MAMBA: open-codestral-mamba
> - MISTRAL: mistral
> - MINISTRAL: ministral
> - PIXTRAL: pixtral
> - UNKNOWN: unknown
> - ANY: typing.Literal['gpt-41', 'gpt-45', 'gpt-4o', 'o1', 'o3', 'o4', 'gpt-4', 'gpt-35', 'r1', 'gemini-1.5-flash', 'gemini-1.5-pro', 'gemini-2.0-flash', 'gemini-2.5-pro', 'claude-3-haiku', 'claude-3-sonnet', 'claude-3-opus', 'claude-3-5-haiku', 'claude-3-5-sonnet', 'claude-3-7-sonnet', 'codestral', 'open-codestral-mamba', 'mistral', 'ministral', 'pixtral', 'unknown']
> - is_claude: <staticmethod(<function ModelFamily.is_claude at 0x000001BBF73BEE60>)>
> - is_gemini: <staticmethod(<function ModelFamily.is_gemini at 0x000001BBF73BF010>)>
> - is_openai: <staticmethod(<function ModelFamily.is_openai at 0x000001BBF73BEF80>)>
> - is_mistral: <staticmethod(<function ModelFamily.is_mistral at 0x000001BBF73BFAC0>)>



### 6. 总结
在本节中，我们学习了如何使用 AutoGen 从零开始构建一个简单的智能体🎉。我们了解了环境搭建、API 密钥的获取与配置，以及如何使用 OpenAI、DeepSeek 和 Qwen 等不同平台的 LLM 来驱动智能体。我们还学习了如何定义工具函数，并将其赋予智能体使用，以及如何通过流式输出将智能体的响应展示在控制台上。通过本节的学习，我们已经掌握了构建单个智能体的基本技能。

下一节：**[Messages-智能体信息传递](../A02-Messages/Messages.ipynb)**

`Message` 是AutoGen框架中一个很基础但又很重要的模块，多智能之间的信息传递以及智能体内部事件状态的获取都需要依赖 `Message` 模块，掌握 `Message` 模块的基础知识，才能更好的理解AutoGen框架的运行机制。


